In [1]:
# import os
# import sys
# sys.path.insert(0, os.path.join('..', 'app'))

In [1]:
cd ../app

/home/din15/Downloads/Program/Parking/app


In [2]:
from lib.data import ParkingData

DATA_DIR = 'data'

data_obj = ParkingData(data_dir=DATA_DIR)

/home/din15/.local/lib/python3.8/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
22/12/11 22:39:29 WARN Utils: Your hostname, din15 resolves to a loopback address: 127.0.1.1; using 192.168.1.239 instead (on interface wlp3s0)
22/12/11 22:39:29 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/12/11 22:39:30 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/12/11 22:39:34 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debu

In [75]:
TRAIN_START_DT = '2020-01-01'
# TRAIN_END_DT = '2020-04-30'
TRAIN_END_DT = '2020-05-31'
TEST_START_DT = '2020-05-01'
TEST_END_DT = '2020-05-31'

EFFECTIVE_SUBQUERY = """ 
       WHEN
                (   
                   (    BR.ToDay{n} >= BR.FromDay{n}
                    AND DAYOFWEEK(SR.ArrivalTime)-1 BETWEEN BR.FromDay{n} AND BR.ToDay{n}
                    )
                OR (    BR.ToDay{n} < BR.FromDay{n}
                    AND (   DAYOFWEEK(SR.ArrivalTime)-1 BETWEEN BR.FromDay{n} AND 6
                         OR DAYOFWEEK(SR.ArrivalTime)-1 BETWEEN 0 AND BR.ToDay{n}
                         )
                    )
                )
          AND FromDay{n} IS NOT NULL
          AND (   TO_TIMESTAMP(DATE_FORMAT(SR.ArrivalTime, "HH:mm:ss"), "HH:mm:ss") BETWEEN StartTime{n} AND CASE WHEN EndTime{n} = TO_TIMESTAMP('00', 'HH') THEN TO_TIMESTAMP('23:59:59', 'HH:mm:ss') ELSE EndTime{n} END
               OR TO_TIMESTAMP(DATE_FORMAT(SR.DepartureTime, "HH:mm:ss"), "HH:mm:ss") BETWEEN StartTime{n} AND CASE WHEN EndTime{n} = TO_TIMESTAMP('00', 'HH') THEN TO_TIMESTAMP('23:59:59', 'HH:mm:ss') ELSE EndTime{n} END)
          AND (PH.holiday IS NULL
               OR BR.EffectiveOnPH{n} = 1)
          THEN BR.{target_col}{n}
"""

BR_COL_SUBQUERY = """
     , CASE  
{effective_subquery}
       END AS {target_col}
"""

_subqueries = []
for col in ['StartTime', 'EndTime', 'Duration', 'EffectiveOnPH', 'DisabilityExt', 'Exemption', 'TypeDesc']:  # 
    _subqueries.append(
        BR_COL_SUBQUERY.format(
            effective_subquery=''.join([EFFECTIVE_SUBQUERY.format(n=n, target_col=col) for n in range(1, 7)]),
            target_col=col
        )
    )

ML_QUERY = """
SELECT * 
     , CASE WHEN LOWER(TypeDesc) LIKE '%loading zone%' THEN 1 ELSE 0 END AS loading_zone
     , CASE WHEN LOWER(TypeDesc) LIKE '%disabled%' THEN 1 ELSE 0 END AS disabled
     , CASE WHEN LOWER(TypeDesc) LIKE '%meter%' THEN 1 ELSE 0 END AS meter
     , CASE WHEN LOWER(TypeDesc) LIKE '%clearway%' THEN 1 ELSE 0 END AS clearway
     , CASE WHEN LOWER(TypeDesc) LIKE '%no stopping%' THEN 1 ELSE 0 END AS no_stopping
     , CASE WHEN LOWER(TypeDesc) LIKE '%no park%' THEN 1 ELSE 0 END AS no_parking
     , CASE WHEN LOWER(TypeDesc) LIKE '%ticket%' THEN 1 ELSE 0 END AS ticket
     , CASE WHEN LOWER(TypeDesc) LIKE '%p%' AND LOWER(TypeDesc) NOT LIKE '%pp%' THEN 1 ELSE 0 END AS letter_p
FROM (


SELECT SR.BayId
     , SR.ArrivalTime
     , SR.DepartureTime
     , SR.DurationMinutes
     , SR.InViolation
     , SR.VehiclePresent
     , CASE WHEN PH.holiday IS NULL THEN 0 ELSE 1 END AS is_holiday
     , DAYOFWEEK(SR.ArrivalTime)-1 AS WeekDay  -- 1=Sun, 7=Sat  >>  0=Sun, 7=Sat
     , CASE WHEN BR.BayId IS NULL THEN 1 ELSE 0 END AS no_br_record
{subqueries}
FROM sr_table SR
LEFT JOIN br_table BR
    ON SR.BayId = BR.BayId
LEFT JOIN ph_table PH
    ON DATE(SR.ArrivalTime) = PH.holiday
WHERE DATE(SR.ArrivalTime) BETWEEN DATE('{start_date}') AND DATE('{end_date}')


)
"""

_query = ML_QUERY.format(
    start_date=TRAIN_START_DT,
    end_date=TRAIN_END_DT,
    subqueries=''.join(_subqueries),
)

data_obj.query(_query, df=False).show()


+-----+-------------------+-------------------+---------------+-----------+--------------+----------+-------+------------+-------------------+-------------------+--------+-------------+-------------+---------+-------------------+------------+--------+-----+--------+-----------+----------+------+--------+
|BayId|        ArrivalTime|      DepartureTime|DurationMinutes|InViolation|VehiclePresent|is_holiday|WeekDay|no_br_record|          StartTime|            EndTime|Duration|EffectiveOnPH|DisabilityExt|Exemption|           TypeDesc|loading_zone|disabled|meter|clearway|no_stopping|no_parking|ticket|letter_p|
+-----+-------------------+-------------------+---------------+-----------+--------------+----------+-------+------------+-------------------+-------------------+--------+-------------+-------------+---------+-------------------+------------+--------+-----+--------+-----------+----------+------+--------+
| 1200|2020-04-28 12:25:28|2020-04-28 12:42:24|             17|          0|       

In [2]:
import csv
import json
import numpy as np
import os
import polars as pl

import glob
import pandas as pd
from random import randrange
from datetime import datetime, timedelta

from lib.data import DataObject, load_json
from config.data import *
from config.graph import *

DATA_DIR = '../app/data'

In [3]:
data_obj = DataObject(data_dir=DATA_DIR)

In [4]:
device_emb = load_json(os.path.join(DATA_DIR, EMB_JSON_NAME))

In [5]:
def random_date(start, end):
    delta = end + timedelta(days=1) - start
    return (start + timedelta(days=randrange(delta.days))).date()

def random_time(start, end):
    delta = end - start
    return (start + timedelta(seconds=randrange(delta.seconds)))

def spark_s_data():
    from pyspark.sql import SparkSession, functions as f

    spark = SparkSession \
        .builder \
        .appName("parking") \
        .getOrCreate()

    g_df = spark.read.csv(os.path.join(DATA_DIR, G_DATA_NAME_CSV), header=True)
    g_df = g_df.select(f.col('bay_id').alias('BayID')).distinct()

    b_df = spark.read.csv(os.path.join(DATA_DIR, B_DATA_NAME), header=True)
    b_df = b_df.select(['BayID', 'DeviceID']).distinct()

    b_df = b_df.join(g_df, on='BayID', how='inner').select(f.col('DeviceID').alias('DeviceId'))

    df = spark.read.csv(os.path.join(DATA_DIR, S_DATA_NAME), header=True)\
            .join(b_df, on='DeviceId', how='inner')\
            .withColumn("ArrivalTime", f.to_timestamp('ArrivalTime', 'MM/dd/yyyy h:m:s a'))\
            .withColumn("DepartureTime", f.to_timestamp('DepartureTime', 'MM/dd/yyyy h:m:s a'))\
            .withColumn("ArrivalDate", f.to_timestamp('DepartureTime', 'MM/dd/yyyy h:m:s a').cast('Date'))
    df = df.toDF(*[col.replace(' ', '') for col in df.columns])
    df.coalesce(1).write.partitionBy('ArrivalDate').mode('overwrite').format("parquet").save(os.path.join(DATA_DIR, S_DATA_TABLE))

# spark_s_data()

In [6]:
b_data = pd.read_csv(os.path.join(DATA_DIR, B_DATA_NAME)).set_index('DeviceID').to_dict('index')

In [7]:
start = datetime(year=2017, month=1, day=1)
end = datetime(year=2017, month=12, day=31)

batch_data_path = glob.glob(os.path.join(DATA_DIR, S_DATA_TABLE, f'ArrivalDate={random_date(start, end)}', '*.parquet'))[0]
batch_data = pd.read_parquet(batch_data_path)

In [8]:
sample_idx = np.random.randint(batch_data.shape[0])
sample_data = batch_data.loc[sample_idx, :]
sample_dt = random_time(sample_data['ArrivalTime'], sample_data['DepartureTime'])

weekday = sample_dt.dayofweek # 0 = Mon, 7 = Sun
duration = sample_data['DurationSeconds']
device_id = sample_data['DeviceId']

is_violation = sample_data['InViolation']
is_use = sample_data['VehiclePresent']

In [9]:
device_id

'21867'

In [31]:
sample_data

DeviceId                         21867
ArrivalTime        2017-06-21 22:06:29
DepartureTime      2017-06-21 23:00:00
DurationSeconds                   3211
StreetMarker                     4527W
Sign                              None
Area                    West Melbourne
StreetId                           839
StreetName                 KING STREET
BetweenStreet1           BATMAN STREET
BetweenStreet2         JEFFCOTT STREET
SideOfStreet                         5
InViolation                      False
VehiclePresent                   False
Name: 339, dtype: object

In [30]:
data_obj.g_data.collect().filter(pl.col('bay_id') == b_data[int(device_id)]['BayID']).to_numpy()

array([['21937',
        'Clarendon Street between Wellington Parade and George Street',
        '11947W', 5264, 'IAW1', 20201223122248, 144.982644328797,
        -37.815125222171]], dtype=object)

# sensor data and geo data are different

In [33]:
b_data[int(device_id)]

{'BayID': 5264,
 'Description1': '3P MTR M-F 10:00-15:00',
 'Description2': 'P10 M-F 1500-1600',
 'Description3': '3P MTR M-F 19:00-20:30',
 'Description4': '3P MTR SAT 7:30-20:30',
 'Description5': nan,
 'Description6': nan,
 'DisabilityExt1': 360,
 'DisabilityExt2': 20.0,
 'DisabilityExt3': 360.0,
 'DisabilityExt4': 360.0,
 'DisabilityExt5': nan,
 'DisabilityExt6': nan,
 'Duration1': 180,
 'Duration2': 10.0,
 'Duration3': 180.0,
 'Duration4': 180.0,
 'Duration5': nan,
 'Duration6': nan,
 'EffectiveOnPH1': 0,
 'EffectiveOnPH2': 0.0,
 'EffectiveOnPH3': 0.0,
 'EffectiveOnPH4': 0.0,
 'EffectiveOnPH5': nan,
 'EffectiveOnPH6': nan,
 'EndTime1': '15:00:00',
 'EndTime2': '16:00:00',
 'EndTime3': '20:30:00',
 'EndTime4': '20:30:00',
 'EndTime5': nan,
 'EndTime6': nan,
 'Exemption1': nan,
 'Exemption2': nan,
 'Exemption3': nan,
 'Exemption4': nan,
 'Exemption5': nan,
 'Exemption6': nan,
 'FromDay1': 1,
 'FromDay2': 1.0,
 'FromDay3': 1.0,
 'FromDay4': 6.0,
 'FromDay5': nan,
 'FromDay6': nan,
 '

In [16]:
device_emb[device_id]

[0.65949547290802,
 0.04236556217074394,
 -0.8195937275886536,
 0.17929166555404663,
 0.9067535400390625,
 0.25302648544311523,
 0.7187158465385437,
 0.2816624641418457,
 -0.029325049370527267,
 -0.42516499757766724,
 -0.9476456046104431,
 0.7799811959266663,
 -1.0594756603240967,
 -0.21646928787231445,
 0.8539930582046509,
 1.1114743947982788,
 -0.22079038619995117,
 0.04981782287359238,
 -0.5824868083000183,
 0.6851041316986084,
 0.607040524482727,
 -0.7498977184295654,
 0.7843365669250488,
 -0.19353941082954407,
 -0.5831809043884277,
 -0.3065613806247711,
 0.7278852462768555,
 -0.002504889154806733,
 -0.5381037592887878,
 0.46480509638786316,
 0.7337797284126282,
 -0.7526929974555969]